### Robust PCA

Given a dataset $a_1,...,a_m$ of random vector samples, Principal Component Analysis (PCA) is done by factoring a covariance matrix $M$ into a product $Q\Sigma Q^T$ (aka diagonalization, Schur Decomposition or Eigenvector Decomposition) and then projecting the data onto the eigenvectors with largest eigenvalues. In essence, what is being done is finding a decomposition:

\begin{equation}
M = L + S
\end{equation}

where $L$ is a low-rank matrix (here, the product $\hat Q \; \hat \Sigma\; \hat Q^T$, where the columns of $Q$ are the $k$ eigenvectors with the largest eigenvalues, and $\Sigma$ is diagonal whose diagonal entries are those eigenvalues), and $S$ is "small". When the $\ell_2$ norm is used to indicate the size of $S$, we arrive at the problem

\begin{equation*}
  \begin{aligned}
    &\text{minimize} && \|M - L\|_2  \\
    &\text{subject to} && \text{rank}(L) \leq k
  \end{aligned}
\end{equation*}

with variable matrix $L$ and data matrix $M$. The solution $L$ will be exactly the solution given above.

However, one problem with this approach is that tweaking the entries of $S$ can produce large differences in the reconstructed value of $L$. To address this, we assume that the matrix $S$ is sparse, and instead consider the problem:

\begin{equation*}
  \begin{aligned}
    &\text{minimize} && \|L\|_* + \lambda \|S\|_1 \\
    &\text{subject to} && L + S = M &&&i = 1,...,m
  \end{aligned}
\end{equation*}

with variables $L$ and $S$ and data matrix $M$. Here, $\| \cdot \|_*$ denotes the nuclear norm, defined as:

\begin{equation}
\|A\|_* = \sum_{i = 1}^{\min{m, n}} \sigma_i(A)
\end{equation}

where $A \in \mathbb{R}^{m \times n}$, and $\sigma_i(A)$ denotes the $i$th (non-negative) singular value of $A$.

Note that $M, L$, and $S$ need not be square.

See the references for a more in-depth explanation of why the nuclear norm is the right choice for $L$.

References:

https://statweb.stanford.edu/~candes/papers/RobustPCA.pdf

https://statweb.stanford.edu/~candes/papers/MatrixCompletion.pdf




In [ ]:
import cvxpy as cp
import numpy as np
import scipy as sp

# setup
problemID = "robust_pca_0"
prob = None
opt_val = None

# Variable declarations

import scipy.sparse as sps

np.random.seed(0)
n = 100
r = 10 # Rank
density = 0.1

L1 = np.random.randn(n,r)
L2 = np.random.randn(r,n)
L0 = L1.dot(L2) # Low rank matrix

S0 = sps.rand(n, n, density) # Sparse matrix w/ Normally distributed entries.
S0.data = 10*np.random.randn(len(S0.data))
M = L0 + S0
lam = 0.1

L = cp.Variable(n, n)
S = cp.Variable(n, n)


# Problem construction

f = cp.norm(L, "nuc") + lam*cp.norm1(S)
C = [L + S == M]

prob = cp.Problem(cp.Minimize(f), C)


# Problem collection

# Single problem collection
problemDict = {
    "problemID" : problemID,
    "problem"   : prob,
    "opt_val"   : opt_val
}
problems = [problemDict]



# For debugging individual problems:
if __name__ == "__main__":
    def printResults(problemID = "", problem = None, opt_val = None):
        print(problemID)
        problem.solve()
        print("\tstatus: {}".format(problem.status))
        print("\toptimal value: {}".format(problem.value))
        print("\ttrue optimal value: {}".format(opt_val))
    printResults(**problems[0])
